# Chapter 7: Data Cleaning and Preparation

In [1]:
import numpy as np
import pandas as pd

## 7.1 Handling Missing Data

For numeric data, pandas uses `NaN` (Not a Number) to represent missing data.

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avacado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avacado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

Python's `None` is also treated as NA in object arrays.

In [5]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 7.1.1 Filtering Out Missing Data

`dropna` can be helpful to filter out missing data. On a Series, it returns the Series with only the non-null data and index values.

In [6]:
from numpy import nan as NA

In [7]:
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

Is equivalent to index slicing on `isnull()`/`notnull()`.

In [8]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, `dropna` by default drops any row containing a missing value.

In [9]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                    [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [10]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing `how='all'` will only drop rows that are all NA.

In [11]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass `axis=1`.

In [12]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [13]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Suppose you want to keep only rows containing a certain number of observations. Use the `thresh` argument.

In [24]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.717804,NaN,NaN
1,-1.328251,NaN,NaN
2,0.674920,NaN,0.326103
3,-1.078086,NaN,2.409767
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


In [25]:
df.dropna()

,0,1,2
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


In [26]:
df.dropna(thresh=2)

,0,1,2
2,0.674920,NaN,0.326103
3,-1.078086,NaN,2.409767
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


`.dropna(thresh=i)` only keeps rows that have at least "i"-non NA values.

### 7.1.2 Filling In Missing Data

For most purposes, use `fillna()` to fill in missing data. Calling `fillna(c)` with a constant replaces missing values with that value.

In [27]:
df.fillna(0)

,0,1,2
0,-0.717804,0.000000,0.000000
1,-1.328251,0.000000,0.000000
2,0.674920,0.000000,0.326103
3,-1.078086,0.000000,2.409767
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


Calling `fillna` with a dict, you can use a different fill value for each column.

In [28]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.717804,0.500000,0.000000
1,-1.328251,0.500000,0.000000
2,0.674920,0.500000,0.326103
3,-1.078086,0.500000,2.409767
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


`fillna` returns a new object, but you can modify in place.

In [29]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.717804,0.000000,0.000000
1,-1.328251,0.000000,0.000000
2,0.674920,0.000000,0.326103
3,-1.078086,0.000000,2.409767
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


Same interpolation methods available for reindexing can be used with `fillna`.

In [32]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.372242,-1.242153,1.326479
1,0.430150,-0.100254,-0.304015
2,-1.512890,NaN,1.737573
3,0.624016,NaN,0.069000
4,0.456371,NaN,NaN
5,-1.095843,NaN,NaN


In [33]:
df.fillna(method='ffill')

# 'ffill' takes the last observed value and fills the next valid NA

,0,1,2
0,0.372242,-1.242153,1.326479
1,0.430150,-0.100254,-0.304015
2,-1.512890,-0.100254,1.737573
3,0.624016,-0.100254,0.069000
4,0.456371,-0.100254,0.069000
5,-1.095843,-0.100254,0.069000


In [34]:
df.fillna(method='ffill', limit=2)

# 'limit=' limits how many values to fill

,0,1,2
0,0.372242,-1.242153,1.326479
1,0.430150,-0.100254,-0.304015
2,-1.512890,-0.100254,1.737573
3,0.624016,-0.100254,0.069000
4,0.456371,NaN,0.069000
5,-1.095843,NaN,0.069000


You can fill in values using the mean.

In [35]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 7.2 Data Transformations

### 7.2.1 Removing Duplicates

In [2]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                    'k2':[1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


`duplicated` returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not.

In [3]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

`drop_duplicates` returns a DataFrame where the duplicated array is False.

In [4]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column.

In [5]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by default keep the 1t observed value combination. `keep='last'` will return the last repeated observed value.

In [6]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 7.2.2 Transforming Data Using a Function or Mapping

In [7]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                                'Pastrami', 'corned beef', 'Bacon',
                                'pastrami', 'honey ham', 'nova lox'],
                    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal each food came from.

In [9]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

First convert each value to lowecase using `str.lower()` then using `map()`.

In [8]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [10]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [11]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

`map()` is a convenient way to perform element-wise transformations.

### 7.2.3 Replacing Values

In [12]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The $-999$ value might be sentinel values for missing data. Use `replace()` to replace these values with NA values (produces a new Series unless `inplace=True`).

In [13]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

To replace multiple values at once, pass a list and then the substitute value.

In [14]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of subs.

In [15]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

Argument can also be a dict.

In [16]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 7.2.4 Renaming Axis Indexes

Axis labels can be similarly transformed and modify in-place.

In [17]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [18]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


To crate a transformed version without modifying the original, use `rename()`.

In [19]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


`rename()` can be used in conjunction with a dict-like object providing new values for a subset of the axis labels.

In [21]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


`rename()` saves you from copying the DataFrame manually and assigning `index` and `columns` attributes. Pass `inplace=True` to modify in-place.

In [23]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 7.2.5 Discretization and Binning

Continuous data is often discretized or separated into "bins".

Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets.

In [24]:
ages = [20, 22, 25, 27,21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

Returns a special `Categorical` object. Treat it like an array of strings indicating the bin name.

In [25]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [26]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [29]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

> Note: `pd.value_counts(cats)` are the bin counts for the result of `pandas.cut`.

Pass `right=False` to change the interval to open/exclusive ')'.

In [30]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

Pass your own bin names by passing a list or array to the `labels` option.

In [31]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

Pass an integer number of bins instead of explicit bin edges to compute equal-length bins based on the min and max values.

In [32]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.007, 0.24], (0.007, 0.24], (0.72, 0.95], (0.48, 0.72], (0.48, 0.72], ..., (0.48, 0.72], (0.24, 0.48], (0.48, 0.72], (0.007, 0.24], (0.72, 0.95]]
Length: 20
Categories (4, interval[float64]): [(0.007, 0.24] < (0.24, 0.48] < (0.48, 0.72] < (0.72, 0.95]]

`qcut()` bins the data based on sample quantiles. `cut()` may not always result in same number of data points.

In [33]:
data = np.random.randn(1000) # Normally distributed
cats = pd.qcut(data, 4) # Cut into quartiles
cats

[(0.651, 2.997], (-0.079, 0.651], (-0.079, 0.651], (0.651, 2.997], (-0.739, -0.079], ..., (-0.739, -0.079], (0.651, 2.997], (-0.079, 0.651], (-0.739, -0.079], (-3.485, -0.739]]
Length: 1000
Categories (4, interval[float64]): [(-3.485, -0.739] < (-0.739, -0.079] < (-0.079, 0.651] < (0.651, 2.997]]

In [34]:
pd.value_counts(cats)

(0.651, 2.997]      250
(-0.079, 0.651]     250
(-0.739, -0.079]    250
(-3.485, -0.739]    250
dtype: int64

You can pass your own quantiles, $0-1$ inclusive.

In [35]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.079, 1.239], (-0.079, 1.239], (-0.079, 1.239], (1.239, 2.997], (-1.323, -0.079], ..., (-1.323, -0.079], (1.239, 2.997], (-0.079, 1.239], (-1.323, -0.079], (-1.323, -0.079]]
Length: 1000
Categories (4, interval[float64]): [(-3.485, -1.323] < (-1.323, -0.079] < (-0.079, 1.239] < (1.239, 2.997]]

### 7.2.6 Detecting and Filtering Outliers

In [5]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.006688,-0.021516,0.008088,-0.001867
std,0.998987,0.972753,1.021195,0.997251
min,-3.073040,-3.700300,-3.208210,-3.292707
25%,-0.641805,-0.698238,-0.660305,-0.653318
50%,-0.017712,0.005064,0.003185,-0.009658
75%,0.689970,0.642288,0.687675,0.681606
max,3.556858,2.975908,3.016571,3.845670


Suppose you wanted to find values in one of the columns exceeding 3 in absolute value.

In [6]:
col = data[2]
col[np.abs(col) > 3]

149    3.016571
458   -3.208210
655   -3.117804
726   -3.023339
Name: 2, dtype: float64

In [7]:
# Select all rows that have a value exceeding 3 or -3
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
0,-0.540773,1.294097,-0.566423,3.170056
1,3.556858,-1.394923,0.177481,-0.515731
149,-0.191596,0.131349,3.016571,-0.754127
167,3.120053,0.302273,1.013239,0.097288
389,-0.939414,-3.700300,1.218438,-0.547450
458,0.368696,-0.132119,-3.208210,-2.205515
655,0.632191,-0.376616,-3.117804,-1.500703
726,2.471347,0.854522,-3.023339,1.733007
821,1.162550,-0.869899,0.329898,-3.292707
882,-0.335404,-3.620619,0.082601,1.706071


To cap values outside the interval -3 to 3.

In [8]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.005990,-0.020195,0.008420,-0.002590
std,0.996282,0.968243,1.020087,0.992884
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.641805,-0.698238,-0.660305,-0.653318
50%,-0.017712,0.005064,0.003185,-0.009658
75%,0.689970,0.642288,0.687675,0.681606
max,3.000000,2.975908,3.000000,3.000000


`np.sign(data)` produces 1 and -1 values based on whether the values in `data` are positive or negative.

In [9]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,1.0
1,1.0,-1.0,1.0,-1.0
2,1.0,-1.0,-1.0,1.0
3,1.0,1.0,1.0,-1.0
4,1.0,1.0,1.0,1.0


### 7.2.7 Permutation and Random Sampling

Use `numpy.random.permutation()` to permute (randomly reorder) a Series or rows in DataFrame. Calling with the length of the axis produces an array of integers indicating the new ordering.

In [11]:
df = pd.DataFrame(np.arange(5*4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([3, 4, 1, 2, 0])

That array can then be used in `iloc` indexing or the equivalent `take()`.

In [12]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [13]:
# .take(sampler) rearranges DF using sampler array as indices

df.take(sampler)

,0,1,2,3
3,12,13,14,15
4,16,17,18,19
1,4,5,6,7
2,8,9,10,11
0,0,1,2,3


Use `sample()` to select a random subset without replacement.

In [14]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
4,16,17,18,19
2,8,9,10,11


Pass `replace=True` to generate a sample with replacement (allow repeat choices).

In [17]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

2   -1
1    7
3    6
3    6
3    6
0    5
2   -1
3    6
1    7
3    6
dtype: int64

### 7.2.8 Computing Indicator/Dummy Variables

Another type of transformation is converting a categorical variable into a "dummy" or "indicator" matrix (ie. 1 is that category, 0 otherwise). For $k$ distinct values, `get_dummies()` would create a matrix or DF with $k$ columns.

In [18]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
pd.get_dummies(df['key'])

# Each value in 'key' is mapped as 1 if it matches that category, 0 otherwise.

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


`get_dummies()` has a prefix argument to add a prefix to columns in the indicator DF which can then be merged with other data.

In [19]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

# 'data1' is on the left, then add the Indicator matrix on right

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


Let's look at the MovieLens 1M dataset, with rows that belong to multiple categories.

In [20]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('Datasets/movielens/movies.dat', sep='::',
                        header=None, names=mnames)
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adding indicator variables for each genre requires a little bit of wrangling. First, extract the list of unique genres in the dataset.

In [21]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

One way to construct Indicator DF is to start with a DF of all 0's. Then iterate through each movie and set entries in each row of `dummies` to 1, using `dummies.columns()` to compute the column indices for each genre.

In [22]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [23]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

Now use `.iloc()` to set values based on these indices.

In [24]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

Then combine this with `movies`.

In [25]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

> Note: For much larger data, write a lower-level function that writes directly to a NumPy array, then wrap the result in a DataFrame.

A useful recipe is to combine `get_dummies()` with a discretization function like `cut()`.

In [26]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [27]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 7.3 String Manipulations

### 7.3.1 String Object Methods

A comma-separated string can be broken into pieces with `split()`.

In [2]:
val = 'a,b,   guido'
val.split(',')

['a', 'b', '   guido']

`split()` is often combined with `strip()` to trim whitespace (including line breaks).

In [4]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

Substrings can be concatenated together with a two-colon delimiter using addition.

In [5]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

A faster, more Pythonic way is to pass a list or tuple to the `join()` on string `'::'`.

In [6]:
'::'.join(pieces)

'a::b::guido'

Python's `in` is best way to detect substrings, though `index` and `find` can also be used.

In [7]:
'guido' in val

True

In [8]:
val.index(',')

1

In [9]:
val.find(':')

-1

`index` raises an `exception` if string isn't found.  
`find` returns `-1` if string isn't found.

In [10]:
val.index(':')

ValueError: substring not found

`count()` returns the number of occurrences of a particular substring.

In [11]:
val.count(',')

2

`replace()` will substitute occurrences of one pattern for another. It can also delete patterns if passing an empty string.

In [12]:
val.replace(',', '::')

'a::b::   guido'

In [13]:
val.replace(',', '')

'ab   guido'

### 7.3.2 Regular Expressions

*Regular expressions* (regex) provide a flexible way to search or match (often more complex) string patterns in text, using Python's `re` module.

The `re` module falls into three categories:
- Pattern matching
- Substitution
- Splitting

Suppose we wanted to split a string with a variable number of whitespace characters (tabs, spaces, newlines). `\s+` describes one or more whitespace characters.

In [14]:
import re

In [17]:
text = "foo     bar\t baz \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

`re.split('\s+', text)` compiles the regular expression then `split()` is called on the passed text.

`re.compile()` forms a reusable regex object.

In [18]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

> Note: To avoid unwanted escaping with `\` in a regular expression, use *raw* string literals like `r'C:\x'` instead of `'C:\\x'`.

> Note: Creating a regex object with `re.compile` is highly recommended for applying the same expression to many strings.

`findall` returns all matches in a string.  
`search` returns only the first match.  
`match` only matches at the beginning of the string.

In [28]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9.%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

`search` returns a special match object for the first email address in the text. For the preceding regex, the match object can only tell us the start and end position of the pattern in the string.

In [29]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [30]:
text[m.start():m.end()]

'dave@google.com'

In [31]:
print(regex.match(text)) # returns None, only match at the start of string

None


`sub` will return a new string with occurrences of the pattern replaced by a new string.

In [32]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



Suppose you wanted to find email addresses and segment each address into 3 components: username, domain name, and domain suffix.

Put parentheses around the parts of the pattern to segment.

In [33]:
pattern = r'([A-Z0-9.%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

`findall` returns a list of tuples when the pattern has groups.

In [34]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

`sub` also has access to groups in each match.  
`\1` corresponds to the first matched group.
`\2` corresponds to the 2nd, and so forth.

In [35]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### 7.3.3 Vectorized String Functions in pandas

In [36]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [37]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

Series has array-oriented methods for string operations that skip NA values, accessed through Series's `str` attribute.

For example, we could check whether each email address has `'gmail'` in it with `str.contains`.

In [38]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Regular expressions can be used too, along with any `re` options like `IGNORECASE`.

In [40]:
pattern = r'([A-Z0-9.%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

Use `str.get` or index into the `str` attribute to do vectorized element retrieval.

In [41]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [42]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

Similarly, you can slice strings using this syntax.

In [43]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

`extract` will return the captured groups of a regular expression as a DataFrame.

In [44]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


## 7.4 Conclusion